In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/participants_data_used_cars/Participants_Data_Used_Cars"))


['Sample_submission.xlsx', 'Data_Train.xlsx', 'Data_Test.xlsx']


In [2]:
# !pip install lightgbm

In [3]:
train=pd.read_excel('../input/participants_data_used_cars/Participants_Data_Used_Cars/Data_Train.xlsx')
test=pd.read_excel('../input/participants_data_used_cars/Participants_Data_Used_Cars/Data_Test.xlsx')
s=pd.read_excel('../input/participants_data_used_cars/Participants_Data_Used_Cars/Sample_submission.xlsx')
train.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [4]:
train=train[train['Kilometers_Driven']<1000000]

In [5]:
# train=train[train.Name.isin(test.Name)]

In [6]:
df=train.append(test,ignore_index=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [7]:
# df

In [8]:
# df['New_Price'].replace(np.nan,'0 Lakh').apply(lambda x : str(x).split(" ")[0]).astype(np.float64).replace(0.00,np.nan)

In [9]:
df['Name']=df['Name'].astype('category')
df['Location']=df['Location'].astype('category')

df['Fuel_Type']=df['Fuel_Type'].astype('category')
df['Transmission']=df['Transmission'].astype('category')
df['Owner_Type']=df['Owner_Type'].astype('category')
df['Mileage']=df['Mileage'].apply(lambda x : str(x).split(" ")[0]).astype(np.float64)
df['Engine']=df['Engine'].apply(lambda x : str(x).split(" ")[0]).astype(np.float64)
df['Power']=df['Power'].replace('null bhp','0 bhp').apply(lambda x : str(x).split(" ")[0]).astype(np.float64)
df['New_Price']=df['New_Price'].replace(np.nan,'0 Lakh').apply(lambda x : str(x).split(" ")[0]).astype(np.float64).replace(0.00,np.nan)
df['Car_Age']=df['Year'].apply(lambda x: 2019-x)
# df['Year']=df['Year'].astype('category')



# df['Engine'].fillna(df['Engine'].mean(),inplace=True)
# df['Mileage'].fillna(df['Mileage'].mean(),inplace=True)
# df['New_Price'].fillna(df['New_Price'].mean(),inplace=True)
# df['Power'].fillna(df['Power'].mean(),inplace=True)
# df['Seats'].fillna(df['Seats'].mean(),inplace=True)

In [10]:
%matplotlib inline
# df.groupby(['Brand','Fuel_Type']).transform(lambda x: x.fillna(x.mean()))['Engine']
# df['Km_per_seats'].plot()
# df['Brand']=df['Name'].apply(lambda x: " ".join(x.split(" ")[:2])).astype('category')
# df.groupby(['Brand','Location']).transform(lambda x: x.fillna(x.mean()))

In [11]:
df['Brand']=df['Name'].apply(lambda x: " ".join(x.split(" ")[:2])).astype('category')
df['Engine']=df.groupby(['Brand']).transform(lambda x: x.fillna(x.mean()))['Engine']
df['Power']=df.groupby(['Brand']).transform(lambda x: x.fillna(x.mean()))['Power']
df['Seats']=df.groupby(['Brand']).transform(lambda x: x.fillna(x.mean()))['Seats']
df['Mileage']=df.groupby(['Brand']).transform(lambda x: x.fillna(x.mean()))['Mileage']
df['New_Price']=df.groupby(['Brand']).transform(lambda x: x.fillna(x.mean()))['New_Price']

In [12]:
df['Engine_bins']=pd.cut(df['Engine'],4,labels=['L','M','H','VH']).astype('category')
# df['Kilometers_Driven']=pd.cut(df['Kilometers_Driven'],5,labels=['L1','ML1','M1','H1','VH1']).astype('category')
# df['Mileage_bins']=pd.cut(df['Mileage'],5,labels=['L2','ML2','M2','H2','VH2']).astype('category')
# df['Power']=pd.cut(df['Power'],4,labels=['L2','M2','H2','VH2']).astype('category')



In [13]:
# df['Brand']=df['Name'].apply(lambda x: x.split(" ")[0])
# df['New_Price']=np.log1p(df['New_Price'])
# df['Mileage_Power']=df['Mileage']*df['Power']
# df['Km_per_seats']=df['Kilometers_Driven']/df['Seats']

In [14]:
df.isnull().sum()
# df['Brand']

Engine                  0
Fuel_Type               0
Kilometers_Driven       0
Location                0
Mileage                 2
Name                    0
New_Price            1511
Owner_Type              0
Power                   0
Price                1234
Seats                   3
Transmission            0
Year                    0
Car_Age                 0
Brand                   0
Engine_bins             0
dtype: int64

In [15]:
# df[(df.Engine.isnull()==True) ]['Brand'].unique()

In [16]:
# df[(df.Engine.isnull()==True) & (df.Brand=='Honda City')]['Engine']

In [17]:
# df['Engine'].isnull().sum()

In [18]:
# df.groupby(['Brand']).transform(lambda x: x.fillna(x.mean()))['Engine']
# agg_cols={'Engine':['mean','sum','min','max'],'Kilometers_Driven':['mean','sum','min','max'],'Mileage':['mean','sum','min','max'],'New_Price':['mean','sum','min','max']
#           ,'Power':['mean','sum','min','max'],'Seats':['mean','sum','min','max'],'Car_Age':['mean','sum','min','max'],'Location':['count','nunique'],'Fuel_Type':['count','nunique'],'Owner_Type':['count','nunique']
#           ,'Engine_bins':['count','nunique']}
agg_cols={'Engine':['mean','sum','min','max'],'Kilometers_Driven':['mean','sum','min','max'],'Mileage':['mean','sum','min','max'],'New_Price':['mean','sum','min','max']
          ,'Power':['mean','sum','min','max'],'Seats':['mean','sum','min','max'],'Car_Age':['mean','sum','min','max'],'Location':['count']}
# agg_cols={'Engine':['mean'],'Mileage':['mean']
#           ,'Power':['mean'],'Location':['count']}

# agg_cols={'Engine':['mean','min','max'],'Kilometers_Driven':['mean'],'Mileage':['mean']
#           ,'Power':['mean','max','min'],'Seats':['mean'],'Car_Age':['mean'],'Location':['count','nunique'],'Fuel_Type':['count','nunique'],'Owner_Type':['count','nunique']
#           ,'Engine_bins':['count','nunique']}

In [19]:
# df.groupby(['Brand','Engine_bins','Owner_Type','Location']).agg(agg_cols)

In [20]:
df1=df.copy()
df1=df.merge(df.groupby(['Brand']).agg(agg_cols).reset_index(),on=['Brand'],how='left')
# df=df.merge(df.groupby('Location').agg(agg_cols),on='Location',how='left')
# df1=df1.merge(df.groupby('Engine_bins').agg(agg_cols),on='Engine_bins',how='left')
# df1=df.merge(df.groupby('Name').agg(agg_cols),on='Name',how='left')

/opt/conda/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3111: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [21]:
df1.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Engine,7252,NaN,NaN,NaN,1615.84,594.576,72,1198,1493,1968,5998
Fuel_Type,7252,5,Diesel,3851,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kilometers_Driven,7252,NaN,NaN,NaN,57810.9,37499.5,171,34000,53404,73000,775000
Location,7252,11,Mumbai,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mileage,7250,NaN,NaN,NaN,18.1419,4.56244,0,15.17,18.16,21.1,33.54
Name,7252,2041,Mahindra XUV500 W8 2WD,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN
New_Price,5741,NaN,NaN,NaN,19.7836,18.7537,1,7.73733,11.63,23.6962,99.92
Owner_Type,7252,4,First,5951,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Power,7252,NaN,NaN,NaN,110.665,55.0931,0,74,91.2,138.03,616
Price,6018,NaN,NaN,NaN,9.47024,11.1659,0.44,3.5,5.64,9.95,160


In [22]:
df1.columns

Index([                     'Engine',                   'Fuel_Type',
                 'Kilometers_Driven',                    'Location',
                           'Mileage',                        'Name',
                         'New_Price',                  'Owner_Type',
                             'Power',                       'Price',
                             'Seats',                'Transmission',
                              'Year',                     'Car_Age',
                             'Brand',                 'Engine_bins',
                  ('Engine', 'mean'),             ('Engine', 'sum'),
                   ('Engine', 'min'),             ('Engine', 'max'),
       ('Kilometers_Driven', 'mean'),  ('Kilometers_Driven', 'sum'),
        ('Kilometers_Driven', 'min'),  ('Kilometers_Driven', 'max'),
                 ('Mileage', 'mean'),            ('Mileage', 'sum'),
                  ('Mileage', 'min'),            ('Mileage', 'max'),
               ('New_Price', 'mean

In [23]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior  
    """ 
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [24]:
# df1['Brand']=pd.factorize(df1['Brand'])[0]
# # df['Year']=pd.factorize(df['Year'])[0]
# df1=pd.get_dummies(df1,columns=['Fuel_Type','Location','Owner_Type','Transmission','Engine_bins'],drop_first=True)
# # df.drop('Name',axis=1,inplace=True)
# df1.head()


In [25]:
df_train = df1[df1['Price'].isnull()==False]
df_test = df1[df1['Price'].isnull()==True]
print(df_train.shape,df_test.shape)

(6018, 45) (1234, 45)


In [26]:
# len(np.setdiff1d(df_train.Brand,df_test.Brand))
# df_train=df_train[df_train.Brand.isin(df_test.Brand)]

columns=['Fuel_Type','Location','Owner_Type','Transmission','Engine_bins','Brand']
for k in columns:
    trn, sub = target_encode(df_train[k], 
                             df_test[k], 
                             target=df_train.Price, 
                             min_samples_leaf=100,
                             smoothing=10,
                             noise_level=0)
    df_train[k]=trn
    df_test[k]=sub
df_train.head(10)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,Name,New_Price,Owner_Type,Power,Price,Seats,Transmission,Year,Car_Age,Brand,Engine_bins,"(Engine, mean)","(Engine, sum)","(Engine, min)","(Engine, max)","(Kilometers_Driven, mean)","(Kilometers_Driven, sum)","(Kilometers_Driven, min)","(Kilometers_Driven, max)","(Mileage, mean)","(Mileage, sum)","(Mileage, min)","(Mileage, max)","(New_Price, mean)","(New_Price, sum)","(New_Price, min)","(New_Price, max)","(Power, mean)","(Power, sum)","(Power, min)","(Power, max)","(Seats, mean)","(Seats, sum)","(Seats, min)","(Seats, max)","(Car_Age, mean)","(Car_Age, sum)","(Car_Age, min)","(Car_Age, max)","(Location, count)"
0,998.0,9.398036,72000,9.429329,26.60,Maruti Wagon R LXI CNG,5.469556,9.951276,58.16,1.75,5.0,5.332703,2010,9,2.845635,4.634522,1015.180791,180702.180791,998.0,1197.0,53742.988764,9566252,600,200000,20.915674,3722.99,12.00,33.54,5.469556,973.580889,4.59,6.80,66.285198,11798.765198,57.50,81.80,5.000000,890.0,5.0,5.0,6.966292,1240,0,18,178
1,1582.0,12.824326,41000,6.726254,19.67,Hyundai Creta 1.6 CRDi SX Option,16.415000,9.951276,126.20,12.50,5.0,5.332703,2015,4,10.368588,16.285602,1572.745763,185584.000000,1396.0,1591.0,40361.211864,4762623,2500,101065,17.673898,2085.52,13.00,22.10,16.415000,1936.970000,11.81,19.18,121.830508,14376.000000,88.70,126.20,5.000000,590.0,5.0,5.0,2.567797,303,0,4,118
2,1199.0,5.701100,46000,7.637505,18.20,Honda Jazz V,8.610000,9.951276,88.70,4.50,5.0,5.332703,2011,8,9.419562,4.634522,1281.246377,89687.246377,1198.0,1498.0,34994.185714,2449593,1015,98000,20.531429,1437.20,0.00,27.30,9.750714,682.550000,8.61,11.15,91.521739,6406.521739,88.70,98.60,5.000000,350.0,5.0,5.0,3.400000,238,0,9,70
3,1248.0,12.824326,87000,7.637505,20.77,Maruti Ertiga VDI,11.384286,9.951276,88.76,6.00,7.0,5.332703,2012,7,9.280184,4.634522,1287.860465,110756.000000,1248.0,1462.0,59245.000000,5095070,12000,168000,20.633953,1774.52,16.02,25.47,11.384286,979.048571,9.54,13.33,89.712093,7715.240000,80.46,103.00,7.000000,602.0,7.0,7.0,4.430233,381,1,7,86
4,1968.0,12.824326,40670,15.078223,15.20,Audi A4 New 2.0 TDI Multitronic,52.976250,7.599886,140.80,17.74,5.0,19.817702,2013,6,10.360696,16.285602,2024.922222,182243.000000,1395.0,3197.0,51382.711111,4624444,11000,125000,16.438111,1479.43,10.50,18.25,52.976250,4767.862500,47.16,55.61,162.041556,14583.740000,0.00,241.40,4.944444,445.0,0.0,5.0,5.888889,530,0,11,90
5,814.0,9.469381,75000,9.776550,21.10,Hyundai EON LPG Era Plus Option,NaN,9.951276,55.20,2.35,5.0,5.332703,2012,7,9.343166,4.634522,825.645570,65226.000000,814.0,998.0,37720.113924,2979889,4000,91000,21.094937,1666.50,20.30,22.00,NaN,0.000000,NaN,NaN,56.003165,4424.250000,55.00,68.05,5.000000,395.0,5.0,5.0,4.240506,335,0,8,79
6,1461.0,12.824326,86999,5.809225,23.08,Nissan Micra Diesel XV,8.528889,9.951276,63.10,3.50,5.0,5.332703,2013,6,9.462709,4.634522,1326.577778,59696.000000,1198.0,1461.0,59539.288889,2679268,6700,123880,20.869556,939.13,18.06,23.19,8.528889,383.800000,6.78,9.51,67.088000,3018.960000,0.00,76.00,5.000000,225.0,5.0,5.0,5.977778,269,1,9,45
7,2755.0,12.824326,36000,9.429329,11.36,Toyota Innova Crysta 2.8 GX AT 8S,21.000000,9.951276,171.50,17.50,8.0,19.817702,2016,3,12.021736,16.285602,2503.261084,508162.000000,1998.0,2755.0,94428.236453,19168932,8899,299322,12.782857,2594.92,11.25,13.68,23.696200,4810.328600,18.64,28.05,117.199507,23791.500000,100.00,171.50,7.433498,1509.0,7.0,8.0,5.650246,1147,0,14,203
8,1598.0,12.824326,64430,6.726254,20.54,Volkswagen Vento Diesel Comfortline,13.296818,9.951276,103.60,5.20,5.0,5.332703,2013,6,6.691981,16.285602,1552.992248,200336.000000,1197.0,1598.0,66225.496124,8543089,3600,720000,18.443101,2379.16,14.40,21.50,13.296818,1715.289545,10.36,16.64,104.194496,13441.090000,103.20,108.50,5.000000,645.0,5.0,5.0,5.868217,757,1,9,129
9,1248.0,12.824326,65932,7.637505,22.30,Tata Indica Vista Quadrajet LS,NaN,7.599886,74.00,1.95,5.0,5.332703,2012,7,9.451746,4.634522,1333.351852,72001.000000,1172.0,1405.0,98763.259259,5333216,8

In [27]:
rem_col=['Name','Price']

In [28]:
# train['Name'].value_counts()
df.columns
# col=['Engine', 'Kilometers_Driven', 'Mileage', 'New_Price', 'Power',
#     'Seats', 'Year', 'Car_Age', 'Brand', 'Fuel_Type_Diesel',
#        'Fuel_Type_Electric', 'Fuel_Type_LPG', 'Fuel_Type_Petrol',
#        'Location_Bangalore', 'Location_Chennai', 'Location_Coimbatore',
#        'Location_Delhi', 'Location_Hyderabad', 'Location_Jaipur',
#        'Location_Kochi', 'Location_Kolkata', 'Location_Mumbai',
#        'Location_Pune', 'Owner_Type_Fourth & Above', 'Owner_Type_Second',
#        'Owner_Type_Third', 'Transmission_Manual', 'Engine_bins_M',
#        'Engine_bins_H', 'Engine_bins_VH']

col=[v for v in list(df1.columns) if v not in rem_col] 
col

['Engine',
 'Fuel_Type',
 'Kilometers_Driven',
 'Location',
 'Mileage',
 'New_Price',
 'Owner_Type',
 'Power',
 'Seats',
 'Transmission',
 'Year',
 'Car_Age',
 'Brand',
 'Engine_bins',
 ('Engine', 'mean'),
 ('Engine', 'sum'),
 ('Engine', 'min'),
 ('Engine', 'max'),
 ('Kilometers_Driven', 'mean'),
 ('Kilometers_Driven', 'sum'),
 ('Kilometers_Driven', 'min'),
 ('Kilometers_Driven', 'max'),
 ('Mileage', 'mean'),
 ('Mileage', 'sum'),
 ('Mileage', 'min'),
 ('Mileage', 'max'),
 ('New_Price', 'mean'),
 ('New_Price', 'sum'),
 ('New_Price', 'min'),
 ('New_Price', 'max'),
 ('Power', 'mean'),
 ('Power', 'sum'),
 ('Power', 'min'),
 ('Power', 'max'),
 ('Seats', 'mean'),
 ('Seats', 'sum'),
 ('Seats', 'min'),
 ('Seats', 'max'),
 ('Car_Age', 'mean'),
 ('Car_Age', 'sum'),
 ('Car_Age', 'min'),
 ('Car_Age', 'max'),
 ('Location', 'count')]

In [29]:

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
v_name = TfidfVectorizer(ngram_range=(1,3),stop_words="english", analyzer='word')
name_tr =v_name.fit_transform(df_train['Name'])
name_ts =v_name.transform(df_test['Name'])


from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
v_namec = TfidfVectorizer(ngram_range=(1,7),stop_words="english", analyzer='char')
name_trc =v_namec.fit_transform(df_train['Name'])
name_tsc =v_namec.transform(df_test['Name'])

In [30]:
# v_name.vocabulary_
df_train[col].head()

,Engine,Fuel_Type,Kilometers_Driven,Location,Mileage,New_Price,Owner_Type,Power,Seats,Transmission,Year,Car_Age,Brand,Engine_bins,"(Engine, mean)","(Engine, sum)","(Engine, min)","(Engine, max)","(Kilometers_Driven, mean)","(Kilometers_Driven, sum)","(Kilometers_Driven, min)","(Kilometers_Driven, max)","(Mileage, mean)","(Mileage, sum)","(Mileage, min)","(Mileage, max)","(New_Price, mean)","(New_Price, sum)","(New_Price, min)","(New_Price, max)","(Power, mean)","(Power, sum)","(Power, min)","(Power, max)","(Seats, mean)","(Seats, sum)","(Seats, min)","(Seats, max)","(Car_Age, mean)","(Car_Age, sum)","(Car_Age, min)","(Car_Age, max)","(Location, count)"
0,998.0,9.398036,72000,9.429329,26.60,5.469556,9.951276,58.16,5.0,5.332703,2010,9,2.845635,4.634522,1015.180791,180702.180791,998.0,1197.0,53742.988764,9566252,600,200000,20.915674,3722.99,12.00,33.54,5.469556,973.580889,4.59,6.80,66.285198,11798.765198,57.50,81.8,5.000000,890.0,5.0,5.0,6.966292,1240,0,18,178
1,1582.0,12.824326,41000,6.726254,19.67,16.415000,9.951276,126.20,5.0,5.332703,2015,4,10.368588,16.285602,1572.745763,185584.000000,1396.0,1591.0,40361.211864,4762623,2500,101065,17.673898,2085.52,13.00,22.10,16.415000,1936.970000,11.81,19.18,121.830508,14376.000000,88.70,126.2,5.000000,590.0,5.0,5.0,2.567797,303,0,4,118
2,1199.0,5.701100,46000,7.637505,18.20,8.610000,9.951276,88.70,5.0,5.332703,2011,8,9.419562,4.634522,1281.246377,89687.246377,1198.0,1498.0,34994.185714,2449593,1015,98000,20.531429,1437.20,0.00,27.30,9.750714,682.550000,8.61,11.15,91.521739,6406.521739,88.70,98.6,5.000000,350.0,5.0,5.0,3.400000,238,0,9,70
3,1248.0,12.824326,87000,7.637505,20.77,11.384286,9.951276,88.76,7.0,5.332703,2012,7,9.280184,4.634522,1287.860465,110756.000000,1248.0,1462.0,59245.000000,5095070,12000,168000,20.633953,1774.52,16.02,25.47,11.384286,979.048571,9.54,13.33,89.712093,7715.240000,80.46,103.0,7.000000,602.0,7.0,7.0,4.430233,381,1,7,86
4,1968.0,12.824326,40670,15.078223,15.20,52.976250,7.599886,140.80,5.0,19.817702,2013,6,10.360696,16.285602,2024.922222,182243.000000,1395.0,3197.0,51382.711111,4624444,11000,125000,16.438111,1479.43,10.50,18.25,52.976250,4767.862500,47.16,55.61,162.041556,14583.740000,0.00,241.4,4.944444,445.0,0.0,5.0,5.888889,530,0,11,90


In [31]:
from scipy.sparse import csr_matrix
from scipy import sparse
final_features = sparse.hstack((df_train[col],name_tr,name_trc )).tocsr()
final_featurest = sparse.hstack((df_test[col],name_ts,name_tsc )).tocsr()

In [32]:
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score,f1_score,mean_squared_error,mean_squared_log_error
X=final_features
y=np.log1p(df_train['Price'].astype(np.float64))
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 1994)

In [33]:
from sklearn.ensemble import AdaBoostRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,RidgeCV,BayesianRidge
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_log_error
import math
def rmsle(real, predicted):
    real=np.expm1(real)
    predicted=np.expm1(predicted)
    return np.sqrt(mean_squared_log_error(real,predicted))
    
def rmsle_lgb(labels, preds):
    return 'rmsle', rmsle(preds,labels), False

In [34]:
from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt 


In [35]:
m=LGBMRegressor(boosting_type='gbdt',
#     num_leaves=255,
#     max_depth=-1,
    learning_rate=0.01,
    n_estimators=6000,
#     subsample_for_bin=200000,
#     objective=None,
#     class_weight=None,
#     min_split_gain=0.0,
#     min_child_weight=0.001,
    min_child_samples=100
                ,
#     subsample=1.0,
#     subsample_freq=1,
    colsample_bytree=0.4,
#     reg_alpha=0.5,
#     reg_lambda=1.0,
    random_state=1994,
    n_jobs=-1,
    silent=True)
m.fit(X_train,y_train,eval_set=[(X_val, y_val.values)],eval_metric='rmse', early_stopping_rounds=100,verbose=100)
p=m.predict(X_val)
print(rmsle_lgb(y_val.values,p))

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's rmse: 0.346405	valid_0's l2: 0.119997
[200]	valid_0's rmse: 0.210544	valid_0's l2: 0.0443288
[300]	valid_0's rmse: 0.165483	valid_0's l2: 0.0273846
[400]	valid_0's rmse: 0.148911	valid_0's l2: 0.0221744
[500]	valid_0's rmse: 0.140611	valid_0's l2: 0.0197715
[600]	valid_0's rmse: 0.135647	valid_0's l2: 0.0184
[700]	valid_0's rmse: 0.132853	valid_0's l2: 0.0176499
[800]	valid_0's rmse: 0.130962	valid_0's l2: 0.0171512
[900]	valid_0's rmse: 0.129744	valid_0's l2: 0.0168336
[1000]	valid_0's rmse: 0.128766	valid_0's l2: 0.0165806
[1100]	valid_0's rmse: 0.12809	valid_0's l2: 0.016407
[1200]	valid_0's rmse: 0.127573	valid_0's l2: 0.0162749
[1300]	valid_0's rmse: 0.127162	valid_0's l2: 0.0161702
[1400]	valid_0's rmse: 0.126784	valid_0's l2: 0.0160743
[1500]	valid_0's rmse: 0.126458	valid_0's l2: 0.0159917
[1600]	valid_0's rmse: 0.126228	valid_0's l2: 0.0159335
[1700]	valid_0's rmse: 0.126005	valid_0's l2: 0.0158772


In [36]:
# m=LGBMRegressor(n_estimators=6000,random_state=1994,learning_rate=0.01)
# m.fit(X_train,y_train,eval_set=[(X_val, y_val.values)],eval_metric='rmse', early_stopping_rounds=100,verbose=100)
# p=m.predict(X_val)
# print(rmsle_lgb(y_val.values,p))

In [37]:
m=LGBMRegressor(n_estimators=m.best_iteration_,random_state=1994,learning_rate=0.01,colsample_bytree=0.4,min_child_samples=100)
m.fit(X,y)
pr=m.predict(final_featurest)

In [38]:
# pr


In [39]:
# np.expm1(pr)

In [40]:
# errlgb=[]
# y_pred_totlgb=[]
# i=0
# from sklearn.model_selection import KFold,StratifiedKFold
# fold=KFold(n_splits=20,shuffle=True,random_state=1994)
# for train_index, test_index in fold.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
#     lgbm_params = {'n_estimators': 6000,
#                    'n_jobs': -1,'learning_rate':0.01,'random_state':1994,'colsample_bytree':0.4}
#     rf=LGBMRegressor(**lgbm_params)
#     rf.fit(X_train,y_train,eval_set=[(X_test, y_test.values)],
#          eval_metric=rmsle_lgb,
#          verbose=200
#          , early_stopping_rounds=200
#           )
#     pr=rf.predict(X_test)
#     print("errlgb: ",rmsle_lgb(y_test.values,pr)[1])
#     errlgb.append(rmsle_lgb(y_test.values,pr)[1])
#     p = rf.predict(final_featurest)
#     y_pred_totlgb.append(p)

In [41]:
# np.mean(errlgb)

In [42]:
# s=pd.DataFrame({'Price':np.expm1(np.mean(y_pred_totlgb,0))})
# s.to_excel('MH-Predict_car_cost_fork_kv2.xlsx',index=False)
# s.head()

In [43]:
s=pd.DataFrame({'Price':np.expm1(pr)})
s.to_excel('MH-Predict_car_cost_fork_kv4.xlsx',index=False)
s.head()

,Price
0,2.717658
1,3.012072
2,15.768604
3,3.819273
4,4.431021
